In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import PorterStemmer,WordNetLemmatizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf


In [4]:
from google.colab import drive
drive.mount('/content/drive/')
#Load data

# df_true = pd.read_csv("/content/drive/My Drive/Uni/IS424/Data/Real_Data.csv")
# df_fake = pd.read_csv("/content/drive/My Drive/Uni/IS424/Data/Fake_Data.csv")
data = pd.read_csv("/content/drive/My Drive/Uni/IS424/new_with sentiment analysis.csv")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
data.head()

,clean_joined,label,vader_final,vader_label,blob_polarity,blob_subjectivity
0,cdc current report death gener discrep death c...,1,-0.9136,negative,-0.125000,0.400000
1,state report death small rise last tuesday sou...,1,-0.8316,negative,-0.083333,0.155556
2,polit correct woman almost use pandem excus re...,0,0.0000,positive,0.000000,0.000000
3,indiafightscorona covid test laboratori india ...,1,0.0000,positive,0.000000,0.000000
4,popul state gener larg case count look new cas...,1,0.0000,positive,0.068182,0.477273


In [ ]:
df_true["labels"], df_fake["labels"] = 1, 0
data=pd.concat([df_true, df_fake],ignore_index=True)
data.head()

,Text,labels
0,The CDC currently reports 99031 deaths. In gen...,1
1,States reported 1121 deaths a small rise from ...,1
2,#IndiaFightsCorona: We have 1524 #COVID testin...,1
3,Populous states can generate large case counts...,1
4,"Covid Act Now found ""on average each person in...",1


In [7]:
texts = data['clean_joined']
labels = data['label']
x=texts
y=labels

# vectorize the text samples into a 2D integer tensor 

MAX_SEQUENCE_LENGTH = 5000
MAX_NUM_WORDS = 25000
EMBEDDING_DIM = 300
TEST_SPLIT = 0.2

In [8]:
# vectorize the text samples into a 2D integer tensor 

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
data = pad_sequences(sequences, 
                     maxlen=MAX_SEQUENCE_LENGTH, 
                     padding='pre', 
                     truncating='pre')

In [9]:
print('Found %s unique tokens.' % len(word_index))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(data, 
                                                  labels.apply(lambda x: 0 if x == 0 else 1), 
                                                  test_size=TEST_SPLIT)

Found 91616 unique tokens.
Shape of data tensor: (51318, 5000)
Shape of label tensor: (51318,)


In [10]:
from keras import layers
from keras.models import Sequential
MAX_SEQUENCE_LENGTH = 5000
MAX_NUM_WORDS = 25000
EMBEDDING_DIM = 300
TEST_SPLIT = 0.2
model = Sequential(
    [
        # part 1: word and sequence processing
        layers.Embedding(num_words,
                         EMBEDDING_DIM, 
                         input_length=MAX_SEQUENCE_LENGTH,
                         trainable=True),
        layers.Conv1D(128, 5, activation='relu'),
        layers.GlobalMaxPooling1D(),
        
        # part 2: classification
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])


model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
history = model.fit(x_train, 
                    y_train,
                    batch_size=100,
                    epochs=10,
                    validation_data=(x_val, y_val))

Epoch 1/10
411/411 [==============================] - 105s 229ms/step - loss: 0.0716 - accuracy: 0.9721 - val_loss: 0.0342 - val_accuracy: 0.9868
Epoch 2/10
411/411 [==============================] - 94s 230ms/step - loss: 0.0191 - accuracy: 0.9926 - val_loss: 0.0283 - val_accuracy: 0.9893
Epoch 3/10
411/411 [==============================] - 94s 229ms/step - loss: 0.0063 - accuracy: 0.9980 - val_loss: 0.0379 - val_accuracy: 0.9887
Epoch 4/10
411/411 [==============================] - 99s 241ms/step - loss: 0.0015 - accuracy: 0.9995 - val_loss: 0.0459 - val_accuracy: 0.9883
Epoch 5/10
411/411 [==============================] - 94s 229ms/step - loss: 2.2146e-04 - accuracy: 0.9999 - val_loss: 0.0618 - val_accuracy: 0.9888
Epoch 6/10
411/411 [==============================] - 94s 229ms/step - loss: 4.9805e-05 - accuracy: 1.0000 - val_loss: 0.0713 - val_accuracy: 0.9888
Epoch 7/10
411/411 [==============================] - 94s 229ms/step - loss: 3.4057e-05 - accuracy: 1.0000 - val_loss: 0.

In [11]:
model.save("/content/drive/My Drive/Uni/IS424/model.pb")

In [ ]:
l, a = model.evaluate(x_val, y_val)
print("Accuracy: " + str(a * 100) + "%")

In [ ]:
history

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.title("Accuracy")
plt.plot(history.history['accuracy'],label='train')
plt.plot(history.history['val_accuracy'],label='test')

**STOP HEREEEEEEEEEEEEEEEEEEEEEEEEEe**
below is outdated

In [ ]:
df_true['category'] = 1
df_false['category'] = 0

In [ ]:
df = pd.concat([df_true,df_false]) 

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
import nltk
nltk.download('stopwords')
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

Stemming

In [ ]:
stemmer = PorterStemmer()
def stem_text(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            word = stemmer.stem(i.strip())
            final_text.append(word)
    return " ".join(final_text)  

In [ ]:
df.Text = df.Text.apply(stem_text)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df.Text,df.category, test_size = 0.2)

In [ ]:
cv=CountVectorizer(min_df=0,max_df=1,ngram_range=(1,2))
#transformed train reviews
cv_train_reviews=cv.fit_transform(x_train)
#transformed test reviews
cv_test_reviews=cv.transform(x_test)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)

In [ ]:
model = Sequential()
model.add(Dense(units = 100 , activation = 'relu' , input_dim = cv_train_reviews.shape[1]))
model.add(Dense(units = 50 , activation = 'relu'))
model.add(Dense(units = 25 , activation = 'relu'))
model.add(Dense(units = 10 , activation = 'relu'))
model.add(Dense(units = 1 , activation = 'sigmoid'))

Compile the model

In [ ]:
model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])

In [ ]:
model.fit(cv_train_reviews, y_train , epochs = 5)